# Trends.Earth API - Script Execution & Monitoring

This notebook focuses on testing GEE script execution and monitoring capabilities of the Trends.Earth API.

## Table of Contents
1. [Setup and Configuration](#setup)
2. [Script Discovery](#discovery)
3. [Script Execution](#execution)
4. [Execution Monitoring](#monitoring)
5. [GEE Script Testing](#gee-testing)

## Setup and Configuration {#setup}

In [ ]:
# Import the shared utilities
import time

from IPython.display import HTML, display
import pandas as pd
from trends_earth_api_utils import (
    SAMPLE_GEOJSON,
    SCRIPT_PARAMS,
    TEST_USERS,
    TrendsEarthAPIClient,
    display_execution_summary,
    get_execution_details,
    get_script_details,
    get_scripts,
    get_user_executions,
    monitor_execution,
    run_script,
    test_all_gee_scripts,
    test_gee_script_execution,
)

# Configuration
API_URL = "http://localhost:5000"  # Update this for your environment

print(f"🌍 Trends.Earth API URL: {API_URL}")
print(f"📋 Available script parameter sets: {list(SCRIPT_PARAMS.keys())}")

# Initialize and login
client = TrendsEarthAPIClient(API_URL)
admin_user = TEST_USERS["admin"]
login_result = client.login(admin_user["email"], admin_user["password"])

if login_result:
    print(f"✅ Logged in as: {admin_user['email']}")
else:
    print("❌ Login failed - check credentials and API connectivity")

## Script Discovery {#discovery}

In [ ]:
# Get list of available scripts
print("📜 Discovering available scripts...")
scripts = get_scripts(client, per_page=50)

if scripts:
    print(f"✅ Found {len(scripts)} scripts")

    # Create scripts overview table
    scripts_df = pd.DataFrame(
        [
            {
                "ID": script.get("id", "")[:8] + "...",
                "Name": script.get("name", "N/A"),
                "Slug": script.get("slug", "N/A"),
                "Status": script.get("status", "Unknown"),
                "Public": "Yes" if script.get("public", False) else "No",
                "Created": script.get("created_at", "")[:10]
                if script.get("created_at")
                else "N/A",
            }
            for script in scripts[:15]
        ]
    )  # Show first 15 scripts

    display(HTML(scripts_df.to_html(index=False)))

    # Script statistics
    statuses = {}
    public_count = 0
    for script in scripts:
        status = script.get("status", "Unknown")
        statuses[status] = statuses.get(status, 0) + 1
        if script.get("public", False):
            public_count += 1

    print("\n📊 Script Statistics:")
    print(f"   Total Scripts: {len(scripts)}")
    print(f"   Public Scripts: {public_count}")
    print(f"   By Status: {dict(statuses)}")
else:
    print("❌ No scripts found")

In [ ]:
# Get details for first few scripts
if scripts:
    print("🔍 Getting detailed information for sample scripts...")

    for i, script in enumerate(scripts[:3]):  # First 3 scripts
        script_id = script.get("id") or script.get("slug")
        print(f"\n📝 Script {i + 1}: {script.get('name', script_id)}")
        print("-" * 40)

        details = get_script_details(client, script_id)
        if details:
            print(f"   ID: {details.get('id', 'N/A')}")
            print(f"   Slug: {details.get('slug', 'N/A')}")
            print(f"   Status: {details.get('status', 'N/A')}")
            print(f"   Public: {details.get('public', False)}")
            print(f"   User ID: {details.get('user_id', 'N/A')}")

            # Show CPU and memory requirements if available
            if "cpu" in details:
                print(f"   CPU: {details['cpu']}")
            if "memory" in details:
                print(f"   Memory: {details['memory']}")
        else:
            print("   ❌ Could not get script details")

        time.sleep(0.5)  # Brief pause

## Script Execution {#execution}

In [ ]:
# Test execution with a simple script
if scripts:
    print("🚀 Testing script execution...")

    # Find a suitable script for testing (prefer public ones)
    test_script = None
    for script in scripts:
        if script.get("public", False) and script.get("status") == "PUBLISHED":
            test_script = script
            break

    if not test_script:
        # Fallback to first available script
        test_script = scripts[0]
        print("⚠️  No public published script found, using first available")

    script_id = test_script.get("id") or test_script.get("slug")
    script_name = test_script.get("name", script_id)

    print(f"📝 Testing with script: {script_name}")

    # Determine appropriate parameters
    script_lower = script_name.lower()
    params = None

    # Try to match script name with our parameter sets
    for param_key, param_set in SCRIPT_PARAMS.items():
        if param_key in script_lower or param_key.replace(
            "-", ""
        ) in script_lower.replace("-", ""):
            params = param_set.copy()
            params["task_name"] = f"test_execution_{int(time.time())}"
            print(f"✅ Using {param_key} parameter set")
            break

    if not params:
        # Use minimal default parameters
        params = {
            "task_name": f"test_execution_{int(time.time())}",
            "task_notes": "API testing execution",
            "geojson": SAMPLE_GEOJSON,
            "year_start": 2018,
            "year_end": 2020,
        }
        print("⚠️  Using minimal default parameters")

    print("📋 Parameters summary:")
    print(f"   Task name: {params.get('task_name', 'N/A')}")
    print(f"   GeoJSON areas: {len(params.get('geojson', params.get('geojsons', [])))}")

    # Execute the script
    execution = run_script(client, script_id, params)

    if execution:
        execution_id = execution.get("id")
        print("✅ Script execution started successfully!")
        print(f"   Execution ID: {execution_id}")
        print(f"   Status: {execution.get('status', 'Unknown')}")
        print(f"   Created: {execution.get('created_at', 'N/A')}")
    else:
        print("❌ Script execution failed")
        execution_id = None
else:
    print("⚠️  No scripts available for testing")
    execution_id = None

## Execution Monitoring {#monitoring}

In [ ]:
# Get current user's executions
print("📊 Getting user's execution history...")

user_executions = get_user_executions(client, per_page=20, sort="-created_at")

if user_executions:
    print(f"✅ Found {len(user_executions)} executions")

    # Display execution summary
    display_execution_summary(user_executions)

    # Execution statistics
    statuses = {}
    total_progress = 0
    progress_count = 0

    for exec in user_executions:
        status = exec.get("status", "Unknown")
        statuses[status] = statuses.get(status, 0) + 1

        if exec.get("progress") is not None:
            total_progress += exec["progress"]
            progress_count += 1

    avg_progress = total_progress / progress_count if progress_count > 0 else 0

    print("\n📈 Execution Statistics:")
    print(f"   Total Executions: {len(user_executions)}")
    print(f"   By Status: {dict(statuses)}")
    print(f"   Average Progress: {avg_progress:.1f}%")
else:
    print("📝 No executions found")

In [ ]:
# Monitor the execution we just started (if any)
if "execution_id" in locals() and execution_id:
    print(f"🔄 Monitoring execution {execution_id}...")

    # Monitor for up to 2 minutes
    final_execution = monitor_execution(client, execution_id, max_wait=120)

    if final_execution:
        print("\n📋 Final Execution Details:")
        print(f"   ID: {final_execution.get('id', 'N/A')}")
        print(f"   Status: {final_execution.get('status', 'N/A')}")
        print(f"   Progress: {final_execution.get('progress', 'N/A')}%")
        print(f"   Created: {final_execution.get('created_at', 'N/A')}")
        print(f"   Updated: {final_execution.get('updated_at', 'N/A')}")

        if final_execution.get("error_message"):
            print(f"   Error: {final_execution['error_message']}")

        if final_execution.get("results"):
            print("   Results available: Yes")
            results = final_execution["results"]
            if isinstance(results, dict):
                print(f"   Result keys: {list(results.keys())}")
    else:
        print("❌ Could not get final execution details")
else:
    print("⚠️  No execution to monitor")

In [ ]:
# Get details for recent executions
if user_executions:
    print("🔍 Getting detailed information for recent executions...")

    for i, exec in enumerate(user_executions[:3]):  # First 3 executions
        exec_id = exec.get("id")
        print(f"\n📊 Execution {i + 1}: {exec_id[:8]}...")
        print("-" * 30)

        details = get_execution_details(client, exec_id)
        if details:
            print(f"   Script ID: {details.get('script_id', 'N/A')}")
            print(f"   Status: {details.get('status', 'N/A')}")
            print(f"   Progress: {details.get('progress', 'N/A')}%")
            print(f"   Start Time: {details.get('start_date', 'N/A')}")
            print(f"   End Time: {details.get('end_date', 'N/A')}")

            # Show parameters if available
            if details.get("params"):
                params = details["params"]
                if isinstance(params, dict):
                    print(
                        f"   Parameter keys: {list(params.keys())[:5]}..."
                    )  # First 5 keys
                    if "task_name" in params:
                        print(f"   Task name: {params['task_name']}")

            # Show logs if available
            if details.get("logs"):
                logs = details["logs"]
                print(
                    f"   Log entries: {len(logs) if isinstance(logs, list) else 'N/A'}"
                )
        else:
            print("   ❌ Could not get execution details")

        time.sleep(0.5)  # Brief pause

## GEE Script Testing {#gee-testing}

In [ ]:
# Test specific GEE scripts with proper parameters
print("🌍 Testing specific GEE scripts...")

# Test productivity script
if scripts:
    productivity_execution = test_gee_script_execution(client, "productivity")

    if productivity_execution:
        print(
            f"📊 Productivity execution started: {productivity_execution.get('id', 'N/A')}"
        )

    time.sleep(2)  # Brief pause between tests

In [ ]:
# Test land cover script
if scripts:
    print("\n🌿 Testing land cover script...")
    land_cover_execution = test_gee_script_execution(client, "land-cover")

    if land_cover_execution:
        print(
            f"📊 Land cover execution started: {land_cover_execution.get('id', 'N/A')}"
        )

    time.sleep(2)  # Brief pause

In [ ]:
# Test soil organic carbon script
if scripts:
    print("\n🏔️  Testing soil organic carbon script...")
    soc_execution = test_gee_script_execution(client, "soil-organic-carbon")

    if soc_execution:
        print(f"📊 SOC execution started: {soc_execution.get('id', 'N/A')}")

    time.sleep(2)  # Brief pause

In [ ]:
# Run comprehensive GEE script testing (limited set)
print("\n🧪 Running comprehensive GEE script testing...")
print("=" * 50)

if scripts:
    gee_test_results = test_all_gee_scripts(client, monitor=False)

    if gee_test_results:
        print("\n📊 GEE Script Testing Results:")

        # Create results summary table
        results_df = pd.DataFrame(
            [
                {
                    "Script": result["script_name"][:30] + "..."
                    if len(result["script_name"]) > 30
                    else result["script_name"],
                    "Execution ID": result["execution_id"][:8] + "..."
                    if result["execution_id"]
                    else "N/A",
                    "Success": "✅" if result["success"] else "❌",
                    "Final Status": result.get("final_status", "Not monitored"),
                }
                for result in gee_test_results
            ]
        )

        display(HTML(results_df.to_html(index=False, escape=False)))

        # Summary statistics
        successful = sum(1 for r in gee_test_results if r["success"])
        print(
            f"\n🎯 Success Rate: {successful}/{len(gee_test_results)} ({(successful / len(gee_test_results) * 100):.1f}%)"
        )
    else:
        print("❌ No GEE test results available")
else:
    print("⚠️  No scripts available for GEE testing")

In [ ]:
# Final execution status check
print("\n🔍 Final execution status check...")

# Get updated user executions
updated_executions = get_user_executions(client, per_page=10, sort="-created_at")

if updated_executions:
    print("📊 Current execution status (last 10):")

    status_summary = {}
    for exec in updated_executions:
        status = exec.get("status", "Unknown")
        status_summary[status] = status_summary.get(status, 0) + 1

    for status, count in status_summary.items():
        print(f"   {status}: {count}")

    # Show any recently completed executions
    completed = [
        e for e in updated_executions if e.get("status") in ["SUCCESS", "FAILED"]
    ]
    if completed:
        print(f"\n✅ {len(completed)} executions completed since testing started")
        for exec in completed[:3]:  # Show first 3
            exec_id = exec.get("id", "")[:8] + "..."
            status = exec.get("status")
            progress = exec.get("progress", "N/A")
            print(f"   {exec_id}: {status} ({progress}%)")
else:
    print("📝 No recent executions found")

print("\n📊 SCRIPT EXECUTION & MONITORING TEST SUMMARY")
print("=" * 60)
print("✅ Script discovery tested")
print("✅ Script execution tested")
print("✅ Execution monitoring tested")
print("✅ GEE script parameters tested")
print("✅ Execution history analysis tested")

# Logout
client.logout()
print("\n🎉 Script Execution and Monitoring tests completed!")